In [9]:
# Código mejorado para analizar días faltantes por mes y año
import json
import os
import calendar
from datetime import datetime
import re

# Ruta al archivo JSON
json_path = "data/processed/datos_electricos_organizados.json"

# Lista de meses en español (en orden)
MESES = ["enero", "febrero", "marzo", "abril", "mayo", "junio", 
         "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

# Diccionario para convertir nombres de meses a números
MES_A_NUMERO = {mes: i+1 for i, mes in enumerate(MESES)}

a=set()
def extraer_dia_de_fecha(fecha_str):
    """
    Extrae el día de una fecha en varios formatos posibles
    """
    if not fecha_str or not isinstance(fecha_str, str):
        return None

    # Limpiar la fecha antes de procesarla
    fecha_str = fecha_str.strip()
    a.add(fecha_str)
    # 1. Formato yyyy-mm-dd HH:MM:SS (con timestamp)
    if re.match(r"^\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}$", fecha_str):
        return int(fecha_str.split("-")[2].split()[0])

    # 2. Formato dd/mm/yyyy o d/m/yyyy
    if re.match(r"^\d{1,2}/\d{1,2}/\d{4}$", fecha_str):
        return int(fecha_str.split("/")[0])

    # 3. Formato yyyy-mm-dd (sin timestamp)
    if re.match(r"^\d{4}-\d{2}-\d{2}$", fecha_str):
        return int(fecha_str.split("-")[2])

    # 4. Formato dd-mm-yyyy
    if re.match(r"^\d{1,2}-\d{1,2}-\d{4}$", fecha_str):
        return int(fecha_str.split("-")[0])

    # 5. Formato texto "DD de Mes de YYYY"
    match = re.match(
        r"^(\d{1,2})(?:\s+de)?\s+\w+(?:\s+de)?\s+\d{4}", fecha_str, re.IGNORECASE
    )
    if match:
        return int(match.group(1))


    return None


def analizar_datos_electricos():
    """Analiza el archivo JSON de datos eléctricos y muestra los meses y días faltantes"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print("=" * 70)
        print(f"📊 ANÁLISIS DE DATOS ELÉCTRICOS ({json_path})")
        print("=" * 70)
        
        # Lista de años disponibles
        años_disponibles = sorted(data.keys())
        print(f"📅 Años disponibles: {', '.join(años_disponibles)}\n")
        
        # Contadores globales
        total_meses_analizados = 0
        meses_con_articulos = 0
        meses_sin_articulos = 0
        meses_incompletos = 0
        dias_faltantes_totales = 0
        
        # Para cada año, analizar meses
        for año in años_disponibles:
            print(f"\n🗓️ AÑO {año}:")
            
            # Obtener meses disponibles
            meses_disponibles = [m for m in MESES if m in data[año]]
            
            # Verificar qué meses faltan
            meses_faltantes = [mes for mes in MESES if mes not in data[año]]
            
            if meses_faltantes:
                print(f"  ❌ Meses faltantes: {', '.join(meses_faltantes)}")
            else:
                print("  ✅ Todos los meses están presentes")
            
            # Analizar días faltantes para cada mes presente
            for mes in meses_disponibles:
                total_meses_analizados += 1
                
                # Obtener número de mes para cálculos de calendario
                num_mes = MES_A_NUMERO[mes]
                
                # Obtener artículos del mes
                articulos = data[año][mes]
                
                # Si no hay artículos, marcar todo el mes como faltante
                if not articulos:
                    print(f"  📂 {mes.capitalize()}: No hay artículos")
                    meses_sin_articulos += 1
                    continue
                
                meses_con_articulos += 1
                
                # Calcular días en el mes
                try:
                    año_int = int(año)
                    total_dias = calendar.monthrange(año_int, num_mes)[1]
                    
                    # Extraer días de los artículos disponibles
                    dias_disponibles = set()
                    articulos_sin_fecha = 0
                    formatos_no_reconocidos = set()
                    
                    for articulo in articulos:
                        if isinstance(articulo, dict) and "fecha" in articulo:
                            fecha_str = articulo["fecha"]
                            dia = extraer_dia_de_fecha(fecha_str)
                            
                            if dia is not None and 1 <= dia <= total_dias:
                                dias_disponibles.add(dia)
                            elif fecha_str:  # Si hay texto pero no se pudo extraer el día
                                formatos_no_reconocidos.add(fecha_str)
                        else:
                            articulos_sin_fecha += 1
                    
                    # Calcular días faltantes
                    dias_faltantes = sorted(set(range(1, total_dias + 1)) - dias_disponibles)
                    dias_faltantes_totales += len(dias_faltantes)
                    
                    # Mostrar resumen
                    if articulos_sin_fecha > 0:
                        print(f"  ⚠️ {mes.capitalize()}: {articulos_sin_fecha} artículos sin fecha")
                    
                    if formatos_no_reconocidos:
                        print(f"  ⚠️ {mes.capitalize()}: Formatos de fecha no reconocidos: {', '.join(list(formatos_no_reconocidos)[:3])}" + 
                              (f" y {len(formatos_no_reconocidos) - 3} más" if len(formatos_no_reconocidos) > 3 else ""))
                    
                    if dias_faltantes:
                        meses_incompletos += 1
                        if len(dias_faltantes) > total_dias - 3:  # Si faltan casi todos los días
                            print(f"  📌 {mes.capitalize()}: Faltan {len(dias_faltantes)} de {total_dias} días ({len(dias_disponibles)} disponibles)")
                        elif len(dias_faltantes) > 10:  # Si faltan muchos días
                            print(f"  📌 {mes.capitalize()}: Faltan {len(dias_faltantes)} días ({', '.join(map(str, dias_faltantes))}")
                        else:  # Si faltan pocos días
                            print(f"  📌 {mes.capitalize()}: Faltan los días {', '.join(map(str, dias_faltantes))}")
                    else:
                        print(f"  ✅ {mes.capitalize()}: Completo - Todos los {total_dias} días tienen datos")
                        
                except ValueError:
                    print(f"  ⚠️ {mes.capitalize()}: Error al procesar (año inválido)")
        
        # Resumen general
        print("\n" + "=" * 70)
        print(f"📋 RESUMEN GLOBAL:")
        print(f"  • Total de meses analizados: {total_meses_analizados}")
        print(f"  • Meses con artículos: {meses_con_articulos}")
        print(f"  • Meses sin artículos: {meses_sin_articulos}")
        print(f"  • Meses con días faltantes: {meses_incompletos}")
        print(f"  • Total de días faltantes: {dias_faltantes_totales}")
        print("=" * 70)
        
    except FileNotFoundError:
        print(f"❗ Error: Archivo no encontrado en {json_path}")
    except json.JSONDecodeError:
        print("❗ Error: No se pudo decodificar el archivo JSON")
    except Exception as e:
        print(f"❗ Error inesperado: {str(e)}")
        import traceback
        traceback.print_exc()

# Ejecutar el análisis
analizar_datos_electricos()

📊 ANÁLISIS DE DATOS ELÉCTRICOS (data/processed/datos_electricos_organizados.json)
📅 Años disponibles: 2022, 2023, 2024, 2025


🗓️ AÑO 2022:
  ✅ Todos los meses están presentes
  📂 Enero: No hay artículos
  📂 Febrero: No hay artículos
  📂 Marzo: No hay artículos
  📂 Abril: No hay artículos
  📂 Mayo: No hay artículos
  📂 Junio: No hay artículos
  📂 Julio: No hay artículos
  📂 Agosto: No hay artículos
  📂 Septiembre: No hay artículos
  📂 Octubre: No hay artículos
  📂 Noviembre: No hay artículos
  📌 Diciembre: Faltan 18 días (1, 4, 9, 10, 15, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31

🗓️ AÑO 2023:
  ✅ Todos los meses están presentes
  📌 Enero: Faltan 13 días (1, 2, 3, 4, 7, 8, 12, 15, 19, 20, 21, 28, 29
  📌 Febrero: Faltan los días 1, 4, 5, 16
  📌 Marzo: Faltan 15 días (1, 4, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30
  📌 Abril: Faltan los días 6, 9, 14, 17, 22, 23
  📌 Mayo: Faltan los días 5, 13, 14, 16, 17, 19
  📌 Junio: Faltan los días 4, 9, 18, 27, 28, 30
  📌 Julio: Fa

In [10]:
import pandas as pd
import calendar
from datetime import datetime
import re
f =set()
# Ruta al archivo CSV
csv_path = "data/raw/afectaciones_electricas_cubadebate_filter_2025.csv"

def mostrar_datos_por_año_mes():
    try:
        # Leer el archivo CSV
        df = pd.read_csv(csv_path, encoding='utf-8')
        
        # Extraer fechas
        fechas_procesadas = []
        for index, row in df.iterrows():
            if 'Fecha' in row:
                fecha_str = row['Fecha']
                f.add(fecha_str)
                try:
                    if re.match(r'\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}', fecha_str):
                        fecha = datetime.strptime(fecha_str, '%Y-%m-%d %H:%M:%S')
                        fechas_procesadas.append({
                            'dia': fecha.day,
                            'mes': fecha.month,
                            'año': fecha.year,
                        })
                except (ValueError, TypeError):
                    pass
        
        # Convertir a DataFrame para facilitar el análisis
        df_fechas = pd.DataFrame(fechas_procesadas)
        
        # Agrupar por año y mes
        print("📊 DATOS DISPONIBLES EN EL CSV:")
        print("=" * 60)
        
        # Mapeo de número de mes a nombre
        meses_nombres = {
            1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
            5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
            9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
        }
        
        # Obtener años ordenados
        años = sorted(df_fechas['año'].unique())
        
        # Para cada año
        for año in años:
            print(f"\nAÑO: {año}")
            
            # Filtrar por año
            df_año = df_fechas[df_fechas['año'] == año]
            
            # Obtener meses ordenados
            meses = sorted(df_año['mes'].unique())
            
            # Para cada mes
            for mes in meses:
                mes_nombre = meses_nombres[mes]
                
                # Filtrar por mes
                df_mes = df_año[df_año['mes'] == mes]
                
                # Obtener días ordenados
                dias = sorted(df_mes['dia'].unique())
                
                # Mostrar los días disponibles
                print(f"  MES: {mes_nombre}")
                print(f"    • Días con datos: {', '.join(map(str, dias))}")
                
        print("\n" + "=" * 60)
        
    except Exception as e:
        print(f"Error al procesar el CSV: {str(e)}")

# Ejecutar el análisis
mostrar_datos_por_año_mes()

📊 DATOS DISPONIBLES EN EL CSV:

AÑO: 2022
  MES: Diciembre
    • Días con datos: 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 17, 23, 30

AÑO: 2023
  MES: Enero
    • Días con datos: 5, 6, 9, 10, 11, 12, 13, 14, 16, 17, 18, 22, 23, 24, 25, 26, 27, 30, 31
  MES: Febrero
    • Días con datos: 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28
  MES: Marzo
    • Días con datos: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 25, 31
  MES: Abril
    • Días con datos: 1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30
  MES: Mayo
    • Días con datos: 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 15, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31
  MES: Junio
    • Días con datos: 1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 29
  MES: Julio
    • Días con datos: 3, 6, 13, 19, 20, 21, 30
  MES: Agosto
    • Días con datos: 2, 3, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 27, 2

In [21]:

print(f-a)

{'2023-01-12 09:03:00', '2024-04-26 11:34:00', '2023-07-06 09:56:00', '2024-06-25 08:40:00', '2025-02-11 21:47:00', '2023-03-04 08:48:00'}
